In [1]:
# Imports 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from keras import backend as K
import matplotlib.pyplot as plt 

In [2]:
# Import data
data = pd.read_csv("../../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

# Testing with 90% training data
train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2,random_state=5)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [3]:
# Model setup

# Attention layer
class peel_the_layer(tf.keras.layers.Layer): 

    def __init__(self):    
        ##Nothing special to be done here
        super(peel_the_layer, self).__init__()

    def build(self, input_shape):
        ##Define the shape of the weights and bias in this layer
        ##This is a 1 unit layer. 
        units=1
        ##last index of the input_shape is the number of dimensions of the prev
        ##RNN layer. last but 1 index is the num of timesteps
        self.w=self.add_weight(name="att_weights", shape=(input_shape[-1], units), initializer="normal") #name property is useful for avoiding RuntimeError: Unable to create link.
        self.b=self.add_weight(name="att_bias", shape=(input_shape[-2], units), initializer="zeros")
        super(peel_the_layer,self).build(input_shape)

    def call(self, x):
        ##x is the input tensor..each word that needs to be attended to
        ##Below is the main processing done during training
        ##K is the Keras Backend import
        e = K.tanh(K.dot(x,self.w)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a

        ##return the ouputs. 'a' is the set of attention weights
        ##the second variable is the 'attention adjusted o/p state' or context
        return a, K.sum(output, axis=1)

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3'
tfhub_handle_preprocess =   'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
reshaped = tf.reshape(net,[-1, 768, 1])
lstm = tf.keras.layers.LSTM(512,return_sequences=True)(reshaped)
a, context = peel_the_layer()(lstm)

dense = tf.keras.layers.Dense(1)(context)
model = tf.keras.Model(text_input, dense)
model.summary()

loss = 'mse'
metrics = [tf.keras.metrics.MeanSquaredError(),tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()]
optimiser = tf.optimizers.Adam(learning_rate= 5e-5)

model.compile(optimizer=optimiser, loss=loss, metrics=metrics)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'sequence_output':  108310273   preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
______________________________________________________________________________________________

In [4]:
# Results setup

results = pd.DataFrame()

In [5]:
def train_test(model, epochs, batch_size, results):
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    history = model.fit(x=train_samples,y=train_labels,batch_size=batch_size,epochs=epochs,verbose=2,callbacks=[callback])

    scores = model.evaluate(x=test_samples,y=test_labels)
    model_label = "Epochs - " + str(epochs) + " Batch Size - " + str(batch_size)

    results[model_label] = scores[1:]

    mse = history.history['mean_squared_error']

    plt.plot(mse)
    plt.title("Model: " + model_label)
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.ylim(bottom=0)
    plt.savefig(model_label + ".jpeg",dpi=100,bbox_inches='tight')

    return results

In [6]:
results = train_test(model,20,32,results)
results = train_test(model,20,64,results)
results.T

Epoch 1/20
176/176 - 3075s - loss: 4.3125 - mean_squared_error: 4.3125 - mean_absolute_error: 1.4268 - mean_absolute_percentage_error: 37.2750
Epoch 2/20
176/176 - 3141s - loss: 0.4510 - mean_squared_error: 0.4510 - mean_absolute_error: 0.5355 - mean_absolute_percentage_error: 15.0909
Epoch 3/20
176/176 - 3123s - loss: 0.2809 - mean_squared_error: 0.2809 - mean_absolute_error: 0.4184 - mean_absolute_percentage_error: 11.5963
Epoch 4/20
176/176 - 3130s - loss: 0.1572 - mean_squared_error: 0.1572 - mean_absolute_error: 0.3091 - mean_absolute_percentage_error: 8.5675
Epoch 5/20
176/176 - 3131s - loss: 0.0923 - mean_squared_error: 0.0923 - mean_absolute_error: 0.2356 - mean_absolute_percentage_error: 6.4402
Epoch 6/20
176/176 - 3127s - loss: 0.0605 - mean_squared_error: 0.0605 - mean_absolute_error: 0.1904 - mean_absolute_percentage_error: 5.2447
Epoch 7/20
176/176 - 3128s - loss: 0.0440 - mean_squared_error: 0.0440 - mean_absolute_error: 0.1631 - mean_absolute_percentage_error: 4.4867
Epo

In [ ]:
results = train_test(model,8,32,results)
results = train_test(model,8,64,results)
results.T

In [ ]:
results = train_test(model,4,32,results)
results = train_test(model,4,64,results)
results.T

In [ ]:
results = train_test(model,3,32,results)
results = train_test(model,3,64,results)
results.T

In [ ]:
results = train_test(model,2,32,results)
results = train_test(model,2,64,results)
results.T

In [ ]:
results = train_test(model,1,32,results)
results = train_test(model,1,64,results)
results.T

In [ ]:
results = results.T
results.to_csv("../../data/random_state_five/bert_italian_sentences_scores.csv")
results